In [98]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import r_regression, RFE, SelectFromModel
from sklearn.linear_model import LassoCV

Ladownie danych 

In [99]:
data = np.loadtxt("eeg.csv", dtype=float, delimiter=',')
features = data[:, :-1]
classes = data[:,-1]

In [100]:
def DownWeighting():
    pass

Zadaeklarowanie klasyfikatorow, metod balansowania oraz selekcji cech

In [101]:
classifiers = {
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'Beyesian': GaussianNB()    
}

In [102]:
balancing_methods = {
    'Undersampling': RandomUnderSampler(),
    'SMOTE': SMOTE(),
    # 'Down-weighting': None
}

eksperyment

In [103]:
results = pd.DataFrame(columns=['classifier', 'balance_method', 'accuracy'])

In [104]:
for balancing_name, balancing_method in balancing_methods.items():

    x_resampled, y_resampled = balancing_method.fit_resample(features, classes)
    x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.1, random_state=50, stratify=y_resampled)
    y_train_weight = np.where(y_train == 1, 0.13, 1 )

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    
    for classifier_name, classifier in classifiers.items():
        classifier.fit(x_train, y_train)

        y_pred  = classifier.predict(x_test)

        accuracy = f1_score(y_test, y_pred)
        results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_index=True)

results

RandomUnderSampler()
[2. 1. 1. ... 2. 2. 1.] [1.   0.13 0.13 ... 1.   1.   0.13]
SMOTE()
[1. 2. 1. ... 2. 1. 2.] [0.13 1.   0.13 ... 1.   0.13 1.  ]


C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2592389132.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_index=True)
C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2592389132.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_index=True)
C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2592389132.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_inde

C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2592389132.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_index=True)
C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2592389132.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_index=True)
C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2592389132.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy}, ignore_inde

,classifier,balance_method,accuracy
0,KNN,Undersampling,0.901786
1,DecisionTree,Undersampling,0.696833
2,Beyesian,Undersampling,0.589286
3,KNN,SMOTE,0.837209
4,DecisionTree,SMOTE,0.645431
5,Beyesian,SMOTE,0.666667


In [105]:
max_indexes = results.groupby('classifier')['accuracy'].idxmax()
results_best_balance = results.loc[max_indexes]
results_best_balance

,classifier,balance_method,accuracy
5,Beyesian,SMOTE,0.666667
1,DecisionTree,Undersampling,0.696833
0,KNN,Undersampling,0.901786


In [106]:
results_with_selector = pd.DataFrame(columns= ['classifier', 'balance_method', 'accuracy', 'selector_method', 'k'])

sprawdzenie selektora -rekurcyjne usuwanie cech od k cech 

In [107]:
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector_name = 'Recursive'
selector_method = lambda k: RFE(estimator= estimator, n_features_to_select=k)



for i, (classifier_name, balancing_name) in results_best_balance[['classifier', 'balance_method']].iterrows():
    classifier = classifiers[classifier_name]
    balancing_method = balancing_methods[balancing_name]

    if balancing_name == 'Down-weighting':
        class_weight = {0: 0.13, 1:1}
        x_resampled, y_resampled = features, classes
    else:
        x_resampled, y_resampled = balancing_method.fit_resample(features, classes)
    
    x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.1, random_state=50, stratify=y_resampled)
    
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)

    for k in range(4, 5):
        selector = selector_method(k)
        selector.fit(x_train, y_train)
        x_selected = selector.transform(x_train)
        x_test_selected = selector.transform(x_test)
        print(len(x_train), len(x_selected))
        if balancing_name == 'Down-weighting':
            classifier.fit(x_selected, y_train, class_weight=class_weight)
        else:
            classifier.fit(x_selected, y_train)

        y_pred  = classifier.predict(x_test_selected)

        accuracy = f1_score(y_test, y_pred)
        results_with_selector = results_with_selector.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy, 'selector_method': selector_name, 'k': k}, ignore_index=True)


# x_selected

14862 14862


C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2439084728.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_with_selector = results_with_selector.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy, 'selector_method': selector_name, 'k': k}, ignore_index=True)


2057 2057


C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2439084728.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_with_selector = results_with_selector.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy, 'selector_method': selector_name, 'k': k}, ignore_index=True)


2057 2057


C:\Users\olcia\AppData\Local\Temp\ipykernel_46400\2439084728.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_with_selector = results_with_selector.append({'classifier': classifier_name, 'balance_method': balancing_name, 'accuracy': accuracy, 'selector_method': selector_name, 'k': k}, ignore_index=True)


sprawdzenie selektora LassoCV

In [108]:
# 'Lasso': lambda k: SelectFromModel(LassoCV(n_features_to_select = k)),

sprawdzenie korelacji liniowej pearsona

In [109]:
    # 'Pearson': lambda k: GaussianNB(n_features_to_select = k),

In [110]:
results_with_selector 

,classifier,balance_method,accuracy,selector_method,k
0,Beyesian,SMOTE,0.666667,Recursive,4
1,DecisionTree,Undersampling,0.657778,Recursive,4
2,KNN,Undersampling,0.645914,Recursive,4


wykresyaccuracy  od k cech dla każdej pary classifier+balance method dla każdego selektora